In [18]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
#Dense RNN or CNN ends fully connected layer
#import model
import time
import pickle
x = pickle.load(open("x.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

from datetime import datetime
import os

#normalize data -> scale
x = x/255.0


In [22]:
#NAME = "Cats-vs-dog-cnn-64x2-{}".format(int(time.time()))
#logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
#tensorboard = TensorBoard(logdir, histogram_freq=1)

# Sets up a timestamped log directory.
# logdir = "logs/text_basics/" + time.time()
# Creates a file writer for the log directory.
# file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the text.
# with file_writer.as_default():

#Deciding best model
#Easy: Number of Layers, Nodes per layer, dense layer at end or not
dense_layers = [0] #1, 2
#dense_layer_size = [512]
layer_sizes = [64] #conv layer 32, 128
conv_layers = [3] #1, 2

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
            print(NAME)
            model = Sequential()
            #CNN layer, convert 2d image into a new 2D dot product image
            #3x3 size kernel
            model.add(Conv2D(layer_size, (3,3), input_shape = x.shape[1:]))
            model.add(Activation("relu"))
            #then, a pooling window, where takes max of 2x2 sliding square
            model.add(MaxPooling2D(pool_size=(2,2)))

            for l in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))
            #third layer, to convert image into 1d array
            model.add(Flatten())
            for l in range(dense_layer):
                model.add(Dense(layer_size)) #dense_layer_size
                model.add(Activation("relu"))
                model.add(Dropout(0.2)) #anti-overfitting

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss="binary_crossentropy", 
                        optimizer="adam",
                        metrics=['accuracy'])

            model.fit(x, y, batch_size=32, epochs=10, validation_split=0.1, callbacks = [tensorboard])

model.save('64x3-CNN.model')

3-conv-64-nodes-0-dense-1672696371
Epoch 1/10
702/702 [==============================] - 52s 72ms/step - loss: 0.6414 - accuracy: 0.6217 - val_loss: 0.5885 - val_accuracy: 0.6866
Epoch 2/10
702/702 [==============================] - 49s 70ms/step - loss: 0.5414 - accuracy: 0.7284 - val_loss: 0.5314 - val_accuracy: 0.7423
Epoch 3/10
702/702 [==============================] - 50s 71ms/step - loss: 0.4944 - accuracy: 0.7603 - val_loss: 0.5167 - val_accuracy: 0.7447
Epoch 4/10
702/702 [==============================] - 49s 70ms/step - loss: 0.4588 - accuracy: 0.7867 - val_loss: 0.4677 - val_accuracy: 0.7816
Epoch 5/10
702/702 [==============================] - 48s 69ms/step - loss: 0.4296 - accuracy: 0.8011 - val_loss: 0.4796 - val_accuracy: 0.7727
Epoch 6/10
702/702 [==============================] - 53s 75ms/step - loss: 0.3995 - accuracy: 0.8180 - val_loss: 0.4564 - val_accuracy: 0.7952
Epoch 7/10
702/702 [==============================] - 52s 75ms/step - loss: 0.3726 - accuracy: 0.8344

INFO:tensorflow:Assets written to: 64x3-CNN.model/assets


INFO:tensorflow:Assets written to: 64x3-CNN.model/assets


In [29]:
import cv2
import tensorflow as tf
CATEGORIES = ["Dog", "Cat"]

def prepare(filepath):
    IMG_SIZE = 50
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

model = tf.keras.models.load_model("64x3-CNN.model")
prediction = model.predict([prepare('Datasets/cat.jpg')])
#prediction = model.predict([prepare('dog.jpg')])
print(CATEGORIES[int(prediction[0][0])])
prediction = model.predict([prepare('Datasets/cat2.jpg')])
print(CATEGORIES[int(prediction[0][0])])


1/1 [==============================] - 0s 70ms/step
Dog
1/1 [==============================] - 0s 23ms/step
Cat
